###################################################
#                                                 #
#              연      습      중                   #
#                                                 #
###################################################

```
practie_1 과 다른 점
1. prompt 를 02_search_from_faiss 와 유사하게 변경하여 확인
 --> 속도가 많이 느려짐
 --> 그래도 조금 더 친근한 멘트들 (추천하는 맥락 및 총평) 이 추가됨
```


In [353]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any, Tuple
import time
import re
from dataclasses import dataclass

embedding_model = OpenAIEmbeddings()
persist_directory = "../.db/faiss"
vectorstore = FAISS.load_local(persist_directory, embedding_model, allow_dangerous_deserialization=True)

# 질의어 및 최대 검색 결과 수 설정
# query = "300만원대 삼성 세탁기"
query = "현대카드 할인하는 LG전자 TV"

MAX_SEARCH_RESULTS = 5

In [354]:
@dataclass
class QueryConditions:
    brands: List[str]
    items: List[str]
    card_discounts: List[str]
    price_conditions: List[Tuple[str, int]]
    price_band_conditions: List[Tuple[str, int, int]]

In [355]:
def analyze_brands(query: str) -> List[str]:
    """브랜드 정보 추출 함수"""
    brand_mapping = {
        'LG': 'LG전자',
        'Lg': 'LG전자',
        'lg': 'LG전자',
        '엘지': 'LG전자',
        '엘쥐': 'LG전자',
        '엘지전자': 'LG전자',
        'Samsung': '삼성전자',
        'samsung': '삼성전자',
        'SAMSUNG': '삼성전자',
        '삼성': '삼성전자',
        'Apple': '애플',
        'apple': '애플',
        'APPLE': '애플',
        'Sony': '소니',
        'sony': '소니',
        'SONY': '소니',
        'Prism': '프리즘코리아',
        'prism': '프리즘코리아',
        'PRISM': '프리즘코리아',
        '프리즘': '프리즘코리아',
        'Lucoms': '루컴즈',
        'lucoms': '루컴즈',
        'LUCOMS': '루컴즈',
        'Nex': '넥스',
        'nex': '넥스',
        'NEX': '넥스',
        'TCL': 'TCL',
        'Xiaomi': '샤오미',
        'xiaomi': '샤오미',
        'XIAOMI': '샤오미',
        'Philips': '필립스',
        'philips': '필립스',
        'PHILIPS': '필립스'
    }
    
    brand_pattern = r'(LG(?:전자)?|삼성(?:전자)?|애플|소니|프리즘(?:코리아)?|루컴즈|넥스|TCL|샤오미|필립스)'
    brands_found = re.findall(brand_pattern, query, re.IGNORECASE)
    
    brands = []
    for brand in brands_found:
        normalized_brand = brand.upper()
        for key, value in brand_mapping.items():
            if key.upper() in normalized_brand:
                if value not in brands:
                    brands.append(value)
                break
    
    return brands

In [356]:
def analyze_items(query: str) -> List[str]:
    """품목 정보 추출 함수"""
    item_pattern = r'\b(TV|냉장고|세탁기|건조기|에어컨|공기청정기)\b'
    matches = re.findall(item_pattern, query, re.IGNORECASE)
    
    item_mapping = {
        'tv': 'TV',
        'Tv': 'TV',
        'tV': 'TV'
    }
    
    return [item_mapping.get(item, item) for item in matches]

In [357]:
def analyze_card_discounts(query: str) -> List[str]:
    """카드할인 정보 추출 함수"""
    discount_pattern = r'(\w+카드)\s*(할인하는|할인되는)'
    return [card for card, _ in re.findall(discount_pattern, query)]

In [358]:
def calculate_price_range(amount: int) -> tuple[int, int]:
    """가격대 범위 계산 함수"""
    # 0이 아닌 마지막 숫자의 위치를 찾은 후, 끝 금액은 해당 위치에 1을 더하는 방식으로 구간대 설정
    if amount == 0:
        return (0, 0)
    
    str_amount = str(amount)
    
    last_non_zero = len(str_amount) - 1
    while last_non_zero >= 0 and str_amount[last_non_zero] == '0':
        last_non_zero -= 1
    
    if last_non_zero < 0:  # 예외처리
        return (0, 0)
    
    start_amount = amount
    
    end_amount_str = list(str_amount)
    end_amount_str[last_non_zero] = str(int(end_amount_str[last_non_zero]) + 1)
    # 마지막 0이 아닌 자리 이후의 모든 숫자를 0으로 초기화
    for i in range(last_non_zero + 1, len(end_amount_str)):
        end_amount_str[i] = '0'
    
    end_amount = int(''.join(end_amount_str))
    
    return (start_amount, end_amount)

def convert_korean_number(number: str, unit: str) -> tuple[int, int]:
    """한국어 숫자 단위 변환 함수"""
    # 적어도 만원 이상의 case 만 대응중
    # 만원 미만의 경우 추후 대응 필요

    multiplier = 1
    if '십' in number:
        multiplier = 10
        number = number.replace('십', '')
    elif '백' in number:
        multiplier = 100
        number = number.replace('백', '')
    elif '천' in number:
        multiplier = 1000
        number = number.replace('천', '')
    
    unit_multiplier = {
        '천': 1000,
        '만': 10000,
        '억': 100000000
    }
    
    base_amount = int(number) * multiplier * unit_multiplier[unit]
    return calculate_price_range(base_amount)

In [359]:
def analyze_price_conditions(query: str) -> Tuple[List[Tuple[str, int]], List[Tuple[str, int, int]]]:
    """가격 조건 분석 함수"""
    price_conditions = []
    price_band_conditions = []
    
    # 구간대 대응
    price_range_pattern = r'((?:\d+)?(?:십|백|천)?(?:\d+)?)\s*(천|만|억)원\s*(이상|이하|초과|미만)'
    price_ranges = re.findall(price_range_pattern, query)
    
    for amount, unit, condition in price_ranges:
        base_amount = convert_korean_number(amount, unit)[0]  # 시작 금액만 사용
        
        if condition == '이상':
            price_conditions.append(('>=', base_amount))
        elif condition == '이하':
            price_conditions.append(('<=', base_amount))
        elif condition == '초과':
            price_conditions.append(('>', base_amount))
        elif condition == '미만':
            price_conditions.append(('<', base_amount))
    
    # 금액대 대응
    price_range_band_pattern = r'((?:\d+)?(?:십|백|천)?(?:\d+)?)\s*(천|만|억)원대'
    price_bands = re.findall(price_range_band_pattern, query)
    
    for amount, unit in price_bands:
        start_amount, end_amount = convert_korean_number(amount, unit)
        price_band_conditions.append(('band', start_amount, end_amount))
    
    return price_conditions, price_band_conditions

In [360]:
def analyze_query(query: str) -> QueryConditions:
    """쿼리 분석 통합 함수"""
    # 각 조건 분석, 추후 추가되는 조건은 추가 필요
    brands = analyze_brands(query)
    items = analyze_items(query)
    card_discounts = analyze_card_discounts(query)
    price_conditions, price_band_conditions = analyze_price_conditions(query)
    
    print(f"\n✅ 브랜드 :: {brands}")
    print(f"\n✅ 아이템 :: {items}")
    print(f"\n✅ 카드할인 :: {card_discounts}")
    print(f"\n✅ 가격 조건 :: {price_conditions}")
    print(f"\n✅ 가격대 :: {price_band_conditions}")
    
    return QueryConditions(
        brands=brands,
        items=items,
        card_discounts=card_discounts,
        price_conditions=price_conditions,
        price_band_conditions=price_band_conditions
    )

In [361]:
def get_effective_price(metadata: dict) -> int:
    """메타데이터에서 실제 적용될 가격을 반환하는 함수"""
    max_benefit_price = int(metadata.get('MAX_BENEFIT_PRICE', 0))
    dscnt_sale_prc = int(metadata.get('DSCNT_SALE_PRC', 0))
    sale_prc = int(metadata.get('SALE_PRC', 0))
    
    return max_benefit_price if max_benefit_price > 0 else (
        dscnt_sale_prc if dscnt_sale_prc > 0 else sale_prc
    )

def check_price_conditions(sale_price: int, price_conditions: list) -> bool:
    """가격 조건 체크 함수"""
    if not price_conditions:
        return True
        
    min_price = 0
    max_price = float('inf')
    
    for operator, amount in price_conditions:
        if operator in ['>=', '>']:
            min_price = max(min_price, amount if operator == '>=' else amount + 1)
        elif operator in ['<=', '<']:
            max_price = min(max_price, amount if operator == '<=' else amount - 1)
    
    return min_price <= sale_price <= max_price

def check_price_band_conditions(sale_price: int, price_band_conditions: list) -> bool:
    """가격대 조건 체크 함수"""
    if not price_band_conditions:
        return True
        
    return any(
        start_amount <= sale_price < end_amount 
        for _, start_amount, end_amount in price_band_conditions
    )

In [362]:
def check_brand_condition(metadata: dict, brands: List[str]) -> bool:
    """브랜드 조건 체크 함수"""
    if not brands:
        return True
    
    return any(
        brand in metadata.get('BRND_NM', '') or 
        brand in metadata.get('GOODS_NM', '')
        for brand in brands
    )

def check_item_condition(metadata: dict, items: List[str]) -> bool:
    """품목 조건 체크 함수"""
    if not items:
        return True
    
    return any(
        item in metadata.get('CATEGORY_NMS', '') or 
        item in metadata.get('ARTC_INFO', '') or
        item in metadata.get('SCH_KWD_NM', '')
        for item in items
    )

def check_card_discount_condition(metadata: dict, card_discounts: List[str]) -> bool:
    """카드할인 조건 체크 함수"""
    if not card_discounts:
        return True
    
    return any(
        card in metadata.get('CARD_DC_NAME', '') or 
        card in metadata.get('CARD_DC_NAME_LIST', '')
        for card in card_discounts
    )

def check_price_condition(metadata: dict, query_parts: QueryConditions) -> bool:
    """가격 조건 체크 함수"""
    if not query_parts.price_conditions and not query_parts.price_band_conditions:
        return True
    
    sale_price = get_effective_price(metadata)
    return (
        check_price_conditions(sale_price, query_parts.price_conditions) or
        check_price_band_conditions(sale_price, query_parts.price_band_conditions)
    )

In [363]:
def check_mandatory_conditions(doc: Any, query_parts: QueryConditions) -> bool:
    """필수 조건 체크 함수 - AND 조건 적용"""
    metadata = doc.metadata
    
    # 모든 조건을 순차적으로 체크
    # 순서는 중요하지 않으나, 추후 조건 추가 시 순서 및 AND//OR 조건 등을 확인해야 함
    conditions = [
        check_brand_condition(metadata, query_parts.brands),
        check_item_condition(metadata, query_parts.items),
        check_card_discount_condition(metadata, query_parts.card_discounts),
        check_price_condition(metadata, query_parts)
    ]
    
    # 모든 조건이 True여야 통과
    return all(conditions)

In [364]:
def calculate_brand_score(metadata: dict, brands: List[str]) -> float:
    """브랜드 매칭 점수 계산 함수"""
    # 브랜드가 정확하게 매칭될 수록 더 높은 점수
    # 상품명에 브랜드명이 들어가 있는 경우 적당한 점수
    score = 0.0
    for brand in brands:
        if brand in metadata.get('BRND_NM', ''):
            score += 5.0
        elif brand in metadata.get('GOODS_NM', ''):
            score += 3.5
    return score

def calculate_item_score(metadata: dict, items: List[str]) -> float:
    """품목 매칭 점수 계산 함수"""
    # 품목정보 매칭이 가장 높은 점수이며, 그 다음으로 카테고리명이 포함되어 있는 경우, 검색키워드가 포함되어 있는 경우 순으로 점수 부여
    score = 0.0
    for item in items:
        if item in metadata.get('ARTC_INFO', ''):
            score += 4.0
        elif item in metadata.get('CATEGORY_NMS', ''):
            score += 3.5
        elif item in metadata.get('SCH_KWD_NM', ''):
            score += 2.5
    return score

def calculate_card_discount_score(metadata: dict, card_discounts: List[str]) -> float:
    """카드할인 매칭 점수 계산 함수"""
    # 카드할인이 직접적으로 명시된 경우 더 높은 점수, 리스트에 포함된 경우 상대적으로 낮은 점수
    score = 0.0
    for card in card_discounts:
        if card in metadata.get('CARD_DC_NAME', ''):
            score += 3.5  # 증가: 카드 직접 매칭
        elif card in metadata.get('CARD_DC_NAME_LIST', ''):
            score += 3.0  # 증가: 카드 리스트 매칭
    return score

In [365]:
def score_document(doc: Any, query_parts: QueryConditions) -> float:
    """문서 스코어링 함수 - 가중치 적용"""
    metadata = doc.metadata
    
    scores = [
        calculate_brand_score(metadata, query_parts.brands),
        calculate_item_score(metadata, query_parts.items),
        calculate_card_discount_score(metadata, query_parts.card_discounts),
    ]
    
    return sum(scores)

In [366]:
def process_batch(batch_data: Tuple[List[Any], List[float]], query_parts: QueryConditions, batch_id: int) -> Tuple[List[Dict[str, Any]], int]:
    """배치 단위 처리 함수"""
    docs, scores = batch_data
    results = []
    
    for doc, vector_score in zip(docs, scores):
        if check_mandatory_conditions(doc, query_parts):
            relevance_score = score_document(doc, query_parts)
            results.append({
                'document': doc,
                'vector_score': vector_score,
                'relevance_score': relevance_score,
                'combined_score': vector_score * 0.3 + relevance_score * 0.7  # 수정: 규칙 기반 점수의 비중을 높임
            })
    
    return results, batch_id

In [367]:
def hybrid_search(query: str, k: int = 5, batch_size: int = 1000, max_concurrent_batches: int = 5) -> List[Dict[str, Any]]:
    """하이브리드 검색 실행 함수 - 대용량 데이터 처리 최적화"""
    start_time = time.time()
    print(f"🔍 검색 시작: {query}")
    
    query_parts = analyze_query(query)
    
    initial_search_size = min(100000, batch_size * 100)
    print(f"📊 초기 검색 크기: {initial_search_size:,}개")
    
    vector_search_start = time.time()
    faiss_results = vectorstore.similarity_search_with_score(query, k=initial_search_size)
    print(f"⚡ 벡터 검색 소요 시간: {time.time() - vector_search_start:.2f}초")
    
    docs = [doc for doc, _ in faiss_results]
    scores = [score for _, score in faiss_results]
    total_batches = (len(docs) + batch_size - 1) // batch_size
    batches = [
        (
            docs[i:i + batch_size],
            scores[i:i + batch_size]
        )
        for i in range(0, len(docs), batch_size)
    ]
    
    print(f"📦 총 {total_batches}개 배치 처리 시작")
    
    all_results = []
    processed_batches = 0
    
    with ThreadPoolExecutor(max_workers=max_concurrent_batches) as executor:
        future_to_batch = {
            executor.submit(process_batch, batch, query_parts, i): i
            for i, batch in enumerate(batches)
        }
        
        for future in as_completed(future_to_batch):
            batch_results, batch_id = future.result()
            all_results.extend(batch_results)
            processed_batches += 1
            
            if processed_batches % 10 == 0:
                print(f"⏳ 진행률: {processed_batches}/{total_batches} 배치 처리 완료")
    
    sorted_results = sorted(all_results, key=lambda x: x['combined_score'], reverse=True)[:k]
    
    print(f"✅ 검색 완료: 총 {len(all_results):,}개 결과 중 상위 {k}개 선택")
    print(f"⌛ 총 소요 시간: {time.time() - start_time:.2f}초")
    
    return sorted_results

In [368]:
def rerank_with_llm(query: str, docs: List[Tuple[Any, float]], max_docs_for_rerank: int = None) -> List[Tuple[Any, float]]:
    """
    LLM을 사용한 재순위화 함수
    
    Args:
        query: 검색 쿼리
        docs: (document, score) 튜플의 리스트
        max_docs_for_rerank: 재순위화할 최대 문서 수 (None이면 자동 계산)
    """
    start_time = time.time()
    print(f"\n🔄 재순위화 시작: {query}")
    
    if not docs:
        print("⚠️ 재순위화할 문서가 없습니다.")
        return []
    
    # 문서 크기 계산 및 제한 설정
    def estimate_tokens(doc: Any) -> int:
        """문서의 대략적인 토큰 수 추정"""
        content = doc.page_content
        metadata_str = str(doc.metadata)
        return (len(content) + len(metadata_str)) // 4

    # GPT-4의 컨텍스트 제한을 고려한 문서 수 계산
    MAX_TOKENS = 8000  # GPT-4 컨텍스트의 안전한 제한값
    TOKENS_FOR_PROMPT = 500  # 프롬프트 자체에 필요한 토큰 예약
    available_tokens = MAX_TOKENS - TOKENS_FOR_PROMPT
    
    # 문서별 토큰 수 계산
    docs_with_tokens = [(doc, score, estimate_tokens(doc)) 
                       for doc, score in docs]
    
    # 동적으로 처리 가능한 문서 수 계산
    total_tokens = 0
    docs_to_process = 0
    for _, _, tokens in docs_with_tokens:
        if total_tokens + tokens > available_tokens:
            break
        total_tokens += tokens
        docs_to_process += 1
    
    # 최소/최대 문서 수 제한 적용
    MIN_DOCS = 5
    MAX_DOCS = 30
    docs_to_process = max(MIN_DOCS, min(docs_to_process, MAX_DOCS))
    
    # 사용자가 지정한 제한이 있다면 적용
    if max_docs_for_rerank is not None:
        docs_to_process = min(docs_to_process, max_docs_for_rerank)
    
    print(f"📊 문서 처리 계획:")
    print(f"- 총 입력 문서: {len(docs)}개")
    print(f"- 처리 가능 문서: {docs_to_process}개")
    print(f"- 예상 토큰 사용량: {total_tokens:,} / {MAX_TOKENS:,}")
    
    # 처리할 문서 선택
    docs_to_rerank = docs[:docs_to_process]
    total_docs = len(docs_to_rerank)
    
    try:
        llm_start = time.time()
        llm = ChatOpenAI(model_name="gpt-4", temperature=0)
        
        # 프롬프트 생성
        prompt = PromptTemplate.from_template(
    """당신은 전자제품 전문 쇼핑몰의 상품 검색 도우미입니다.
아래 제공된 상품 정보를 기반으로 고객의 질문에 답변해주세요.

[답변 작성 지침]
1. 고객이 이 상품들을 찾게 된 맥락과 추천 이유를 먼저 설명해주세요.
   - 고객의 검색 의도와 필요성을 분석하여 설명
   - 추천하는 상품들이 어떻게 고객의 요구사항을 충족시키는지 설명
   - 특별한 할인이나 혜택이 있다면 강조

2. 상품 정보는 다음 순서로 정확하게 표기해주세요:
   
   📌 [상품명]
   - 상품상태: [상태]
   - 브랜드: [브랜드명]
   - 정상가: [판매가]원
   - 할인가: [할인가]원
   - 최대혜택가: [최종가격]원
   - 카드할인: [할인카드 목록]
   - 주요 기능:
     ∙ [특징1]
     ∙ [특징2]
     ∙ [특징3]
   🔗 구매하기: https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=[상품번호]
   ───────────────────

3. 가격 정보는 모두 쉼표(,)를 포함한 숫자 형식으로 표기해주세요.
4. 상품 간 구분은 위의 구분선을 사용해주세요.
5. 마지막에는 전체 상품에 대한 간단한 총평을 덧붙여주세요.
   - 추천된 상품들의 전반적인 특징과 장점
   - 가격대비 성능 분석
   - 구매 시 고려해야 할 주요 사항
   - 최종 구매 추천

[전체 검색 결과]
{total_docs}

[추천 상품 목록]
{docs}

[답변]
"""
)

        combined = "\n---\n".join(
            f"문서 {i+1}:\n제목: {doc.metadata.get('GOODS_NM', '정보 없음')}\n내용: {doc.page_content}"
            for i, (doc, _) in enumerate(docs_to_rerank)
        )
        
        chain = (
            prompt
            | llm
            | StrOutputParser()
        )
        response = chain.invoke({"query": query, "docs": combined, "total_docs": total_docs})
        print(f"⚡ LLM 처리 소요 시간: {time.time() - llm_start:.2f}초")
        
        # 응답 파싱 및 검증
        parsing_start = time.time()
        try:
            indices = [int(i.strip()) - 1 for i in response.strip().split(",") if i.strip().isdigit()]
            
            # 검증
            if len(indices) != total_docs:
                print(f"⚠️ 경고: LLM이 {total_docs}개가 아닌 {len(indices)}개의 문서를 반환했습니다.")
                # 누락된 인덱스 찾기
                used_indices = set(indices)
                all_indices = set(range(total_docs))
                missing_indices = all_indices - used_indices
                
                # 누락된 인덱스 추가
                indices.extend(sorted(missing_indices))
                print(f"🔄 누락된 문서 {len(missing_indices)}개를 마지막에 추가했습니다.")
            
            # 중복 체크
            if len(set(indices)) != len(indices):
                print("⚠️ 경고: 중복된 인덱스가 발견되었습니다.")
                indices = list(dict.fromkeys(indices))  # 중복 제거
                # 누락된 인덱스 추가
                all_indices = set(range(total_docs))
                missing_indices = all_indices - set(indices)
                indices.extend(sorted(missing_indices))
            
            # 범위 검증
            indices = [i for i in indices if 0 <= i < total_docs]
            
        except Exception as parse_error:
            print(f"⚠️ 응답 파싱 오류: {parse_error}")
            indices = list(range(total_docs))
        
        # 결과 로깅
        print("\n🔍 재순위화 결과:")
        for rank, idx in enumerate(indices, 1):
            doc = docs_to_rerank[idx][0]
            goods_no = doc.metadata.get('GOODS_NO', '정보 없음')
            goods_nm = doc.metadata.get('GOODS_NM', '정보 없음')
            print(f"순위 {rank}. 상품번호: {goods_no} - {goods_nm}")
        
        # 재순위화된 문서 생성
        reranked = [docs_to_rerank[i] for i in indices]
        
        # 나머지 문서 추가
        if len(docs) > docs_to_process:
            remaining_docs = docs[docs_to_process:]
            print(f"📎 나머지 {len(remaining_docs):,}개 문서 추가")
            reranked.extend(remaining_docs)
        
        print(f"⚡ 결과 파싱 소요 시간: {time.time() - parsing_start:.2f}초")
        print(f"✅ 재순위화 완료: 총 {len(reranked):,}개 결과")
        print(f"⌛ 총 소요 시간: {time.time() - start_time:.2f}초")
        
        return reranked
        
    except Exception as e:
        print(f"❌ 재순위화 중 오류 발생: {e}")
        print(f"⌛ 오류까지 소요 시간: {time.time() - start_time:.2f}초")
        return docs  # 오류 발생 시 원본 순서 유지

In [369]:
def print_search_results(results: List[Dict[str, Any]], title: str = "검색 결과"):
    """검색 결과 출력 함수"""
    print(f"\n📦 {title}")
    for i, result in enumerate(results, 1):
        print(f"[결과 {i}] 최종점수: {result['combined_score']:.4f}")
        doc = result['document']
        print(f"{doc.page_content}")
        try:
            print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
            print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
            print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
            print(f"- 품목정보: {doc.metadata.get('ARTC_INFO', '정보 없음')}")
            print(f"- 카테고리: {doc.metadata.get('CATEGORY_NMS', '정보 없음')}")
            print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
            print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
            print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
            print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
            print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
            print(f"- 주요 특징 및 기능:")
            feature_values = doc.metadata.get('OPT_VAL_DESC', '').split(',')
            feature_titles = doc.metadata.get('OPT_DISP_NM', '').split(',')
            for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                print(f"  - {title} : {value}")
                if i >= 2: break
            print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
        except Exception as e:
            print(f"메타데이터 출력 중 오류: {str(e)}")
        print("-" * 60)

In [370]:
def print_search_results2(results: List[Dict[str, Any]], title: str = "검색 결과", query: str = None):
    """검색 결과 출력 함수 (맥락 설명과 총평 포함)"""
    print(f"\n📦 {title}")
    
    # LLM 응답 출력 (맥락 설명 및 총평)
    if query and len(results) > 0:
        try:
            llm = ChatOpenAI(model_name="gpt-4", temperature=0)
            prompt = PromptTemplate.from_template(
                """다음 검색 결과들에 대해 맥락 설명과 총평을 제공해주세요.

[검색어]
{query}

[검색 결과]
{docs}

1. 맥락 설명:
- 고객의 검색 의도와 필요성을 분석
- 추천하는 상품들이 어떻게 고객의 요구사항을 충족시키는지 설명
- 특별한 할인이나 혜택이 있다면 강조

2. 총평:
- 추천된 상품들의 전반적인 특징과 장점
- 가격대비 성능 분석
- 구매 시 고려해야 할 주요 사항
- 최종 구매 추천
"""
            )
            
            docs_text = "\n\n".join(
                f"상품 {i+1}:\n" + 
                f"- 상품명: {doc['document'].metadata.get('GOODS_NM', '정보 없음')}\n" +
                f"- 가격: {format(int(doc['document'].metadata.get('SALE_PRC', 0)), ',')}원\n" +
                f"- 할인가: {format(int(doc['document'].metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원\n" +
                f"- 주요 특징: {doc['document'].page_content}"
                for i, doc in enumerate(results)
            )
            
            chain = (
                prompt
                | llm
                | StrOutputParser()
            )
            context_and_summary = chain.invoke({"query": query, "docs": docs_text})
            print("\n" + context_and_summary + "\n")
            print("=" * 80 + "\n")
        except Exception as e:
            print(f"LLM 응답 생성 중 오류 발생: {str(e)}\n")
    
    # 개별 상품 정보 출력
    for i, result in enumerate(results, 1):
        print(f"[결과 {i}] 최종점수: {result['combined_score']:.4f}")
        doc = result['document']
        print(f"{doc.page_content}")
        try:
            print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
            print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
            print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
            print(f"- 품목정보: {doc.metadata.get('ARTC_INFO', '정보 없음')}")
            print(f"- 카테고리: {doc.metadata.get('CATEGORY_NMS', '정보 없음')}")
            print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
            print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
            print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
            print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
            print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
            print(f"- 주요 특징 및 기능:")
            feature_values = doc.metadata.get('OPT_VAL_DESC', '').split(',')
            feature_titles = doc.metadata.get('OPT_DISP_NM', '').split(',')
            for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                print(f"  - {title} : {value}")
                if i >= 2: break
            print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
        except Exception as e:
            print(f"메타데이터 출력 중 오류: {str(e)}")
        print("-" * 60)

In [371]:
# 1. 하이브리드 검색 실행
hybrid_results = hybrid_search(
    query=query,
    k=MAX_SEARCH_RESULTS,
    batch_size=1000,  # 배치 크기
    max_concurrent_batches=5  # 동시 처리할 최대 배치 수
)

print_search_results(hybrid_results, "하이브리드 검색 결과")

🔍 검색 시작: 현대카드 할인하는 LG전자 TV

✅ 브랜드 :: ['LG전자']

✅ 아이템 :: ['TV']

✅ 카드할인 :: ['현대카드']

✅ 가격 조건 :: []

✅ 가격대 :: []
📊 초기 검색 크기: 100,000개
⚡ 벡터 검색 소요 시간: 0.70초
📦 총 4개 배치 처리 시작
✅ 검색 완료: 총 216개 결과 중 상위 5개 선택
⌛ 총 소요 시간: 0.71초

📦 하이브리드 검색 결과
[결과 1] 최종점수: 9.0084
LG전자의 OLED TV, 163cm 올레드 4K TV OLED65A3MNA 스탠드형. 주요 특징: 크기: 65인치(163cm), 종류: OLED, 해상도: UHD(4K), 기본 주사율: 60Hz, 엔진: 알파7칩셋, 스마트기능: 넷플릭스, 유튜브, 음성인식, 아마존프라임비디오, 에어플레이2, Disney+, 부가기능: USB재생, 부가기능: 블루투스, 부가기능: 게임모드, 부가기능: HDMI eARC, e효율등급: 4등급, 출시년도: 2023년, 화면 타입: 평면형, 스피커 출력: 20W, 음향효과: 공간인식사운드, 음향효과: 인공지능사운드, 음향효과: 돌비애트모스, 지원단자: HDMI, 지원단자: RF, 지원단자: LAN, 지원단자: Optical. 카테고리: TV·영상가전 > TV > 대형(~189cm). 가격 정보: 판매가 3,034,900원, 롯데카드 카드 사용 시 5% 할인되어 최대 2,883,150원. 가전제품입니다. 해시태그: #OLED65A3MNA_SS.
- 상품상태: 정상상품
- 브랜드명: LG전자
- 상품명: 163cm 올레드 4K TV OLED65A3MNA 스탠드형
- 품목정보: TV>OLED TV
- 카테고리: TV·영상가전>TV>대형(~189cm)
- 판매가: 3,034,900원
- 할인가: 3,034,900원
- 최대혜택가: 2,883,150원
- 카드할인율: 5%
- 할인카드: 롯데카드,현대카드,네이버페이,삼성카드,신한카드,KB국민카드
- 주요 특징 및 기능:
  - 크기 : 65인치(163

In [372]:
# 2. LLM 재순위화
reranked_results = rerank_with_llm(
    query=query,
    docs=[(result['document'], result['combined_score']) for result in hybrid_results],
    max_docs_for_rerank=MAX_SEARCH_RESULTS
)

llm_results = [
    {
        'document': doc,
        'combined_score': score,
        'vector_score': 0,
        'relevance_score': score
    }
    for doc, score in reranked_results
]

print_search_results2(llm_results, "하이브리드 + LLM 재순위화 결과", query)



🔄 재순위화 시작: 현대카드 할인하는 LG전자 TV
📊 문서 처리 계획:
- 총 입력 문서: 5개
- 처리 가능 문서: 5개
- 예상 토큰 사용량: 1,706 / 8,000
⚡ LLM 처리 소요 시간: 41.89초
⚠️ 경고: LLM이 5개가 아닌 15개의 문서를 반환했습니다.
🔄 누락된 문서 5개를 마지막에 추가했습니다.
⚠️ 경고: 중복된 인덱스가 발견되었습니다.

🔍 재순위화 결과:
순위 1. 상품번호: 0021401498 - 163cm 올레드 4K TV OLED65A3MNA 스탠드형
순위 2. 상품번호: 0020454730 - 163cm 올레드 TV OLED65B3FNA 스탠드형
순위 3. 상품번호: 0017120523 - 163cm  올레드 TV 65ART90EKPB (레드우드)
순위 4. 상품번호: 0017120819 - 163cm  올레드 TV 65ART90EKPC (그린)
순위 5. 상품번호: 0035750022 - 163cm LG전자 울트라 HD TV 65UT93RC0NA (스탠드형)
⚡ 결과 파싱 소요 시간: 0.00초
✅ 재순위화 완료: 총 5개 결과
⌛ 총 소요 시간: 41.89초

📦 하이브리드 + LLM 재순위화 결과

1. 맥락 설명:
검색어 "현대카드 할인하는 LG전자 TV"는 현대카드를 사용하여 할인 받을 수 있는 LG전자의 TV를 찾고자 하는 고객의 의도를 반영하고 있습니다. 검색 결과로 나온 상품들은 모두 LG전자의 TV로, 크기는 163cm(65인치)로 동일하며, 주로 OLED와 UHD(4K) 해상도를 지원하는 모델들입니다. 각 상품들은 스마트기능, 부가기능, 음향효과 등 다양한 특징을 가지고 있으며, 롯데카드를 사용할 경우 5%에서 10%까지의 할인 혜택을 받을 수 있습니다.

2. 총평:
검색된 상품들은 모두 LG전자의 고해상도 TV로, 크기나 해상도, 스마트기능 등 주요 사양이 우수한 편입니다. 가격대는 1,140,000원에서 9,400,000원까지 다양하며, 이에 따라 제공되는 기능과 성능이 차이가 있습니다. 예를 